# Ambiente virtual

## Criação

### Em um terminal já na pasta do projeto execute o seguinte comando:
#### Linux:
##### python3 -m venv venv
#### Windows:
##### python -m venv venv

## Ativação

### Linux 
#### source venv/bin/activate
### Windows 
#### venv\Scripts\Activate
### Caso algum comando retorne um erro de permissão execute o código e tente novamente: 
#### Set-ExecutionPolicy -Scope CurrentUser -ExecutionPolicy RemoteSigned

# Bibliotecas

##### pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

# Credenciais

##### 1- Acesse a Console de Desenvolvedores do Google. 
##### 2- Crie um novo projeto. 
##### 3- Ative a API do Google Drive para o projeto.
##### 4- Crie credenciais de serviço e faça o download do arquivo JSON. 
##### 5- Coloque o arquivo JSON na pasta do projeto. 

# Código

## Listagem

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Configurar credenciais
credentials = service_account.Credentials.from_service_account_file('caminho/para/arquivo.json', scopes=['https://www.googleapis.com/auth/drive'])

# Criar serviço do Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# Exemplo: listar arquivos
results = drive_service.files().list().execute()
files = results.get('files', [])

if not files:
    print('Nenhum arquivo encontrado.')
else:
    print('Arquivos:')
    for file in files:
        print(f"{file['name']} ({file['id']})")

## Envio

In [ ]:
# Importar bibliotecas necessárias
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Configurar credenciais
credentials = service_account.Credentials.from_service_account_file('caminho/para/arquivo.json', scopes=['https://www.googleapis.com/auth/drive'])

# Criar serviço do Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

def upload_arquivo(caminho_do_arquivo, nome_do_arquivo_no_drive):
    # Criar um objeto de mídia para upload
    media = MediaFileUpload(caminho_do_arquivo, resumable=True)

    # Parâmetros para a solicitação de upload
    request = drive_service.files().create(
        media_body=media,
        body={'name': nome_do_arquivo_no_drive}
    )

    # Executar a solicitação de upload
    response = None
    while response is None:
        status, response = request.next_chunk()
        if status:
            print(f"Progresso do Upload: {int(status.progress() * 100)}%")

    print(f"Upload do arquivo '{nome_do_arquivo_no_drive}' concluído.")

# Exemplo de uso
upload_arquivo('caminho/do/arquivo/local.txt', 'arquivo_no_drive.txt')

## Download 

In [ ]:
# Importar bibliotecas necessárias
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io

# Configurar credenciais
credentials = service_account.Credentials.from_service_account_file('caminho/para/arquivo.json', scopes=['https://www.googleapis.com/auth/drive'])

# Criar serviço do Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

def download_arquivo(id_do_arquivo, caminho_de_destino):
    try:
        # Fazer uma solicitação para obter informações sobre o arquivo
        arquivo = drive_service.files().get(fileId=id_do_arquivo).execute()

        # Preparar o stream para o download
        request = drive_service.files().get_media(fileId=id_do_arquivo)
        fh = io.FileIO(caminho_de_destino, 'wb')
        downloader = MediaIoBaseDownload(fh, request)

        # Executar o download
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Progresso do Download: {int(status.progress() * 100)}%")

        print(f"Download do arquivo '{arquivo['name']}' concluído. Salvo em '{caminho_de_destino}'.")

    except Exception as e:
        print(f"Erro ao fazer download do arquivo: {e}")

# Exemplo de uso (certifique-se de substituir 'ID_DO_ARQUIVO' pelo ID real do arquivo no Google Drive)
download_arquivo('ID_DO_ARQUIVO', 'caminho/do/arquivo/local_baixado.txt')

## Deleção

In [ ]:
def deletar_arquivo(id_do_arquivo):
    try:
        # Executar a solicitação de deleção
        drive_service.files().delete(fileId=id_do_arquivo).execute()
        print(f"Arquivo com ID {id_do_arquivo} deletado com sucesso.")
    except Exception as e:
        print(f"Erro ao deletar arquivo: {e}")

# Exemplo de uso (certifique-se de substituir 'ID_DO_ARQUIVO' pelo ID real do arquivo no Google Drive)
deletar_arquivo('ID_DO_ARQUIVO')